In [ ]:
import os
import sys
sys.path.append('C:\\Users\\tiwar\\Desktop\\Project\\ACL_Thesis\\Thesis\\point_cloud_filtering\\Utilities')

import open3d as o3d
from Utilities import csf, cloth, particle, rasterization, point_cloud
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
wd = os.getcwd()
print("working directory", wd)
dataset_dir = wd + '\..\..\Datasets\marketplacefeldkirch_station4_intensity_rgb\marketplacefeldkirch_station4_intensity_rgb.txt'      # WINDOWS
print(">>>>>>>> DATA set directory.....\n", dataset_dir)

In [ ]:
# Visualizing the input point cloud
data = point_cloud.point_cloud_formatting(dataset_dir)
pcd_points = point_cloud.point_cloud_data(data.o3d_format_from_file())


# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(projection='3d')

# ax.scatter(pcd_points[:, 0], pcd_points[:, 1], pcd_points[:, 2], marker = '.')
# plt.show()

In [ ]:
# Visualizing the pcd in o3d
o3d.visualization.draw_geometries([pcd_points.pcd])

In [ ]:
# Inverting Point Cloud
inverted_pcd =  point_cloud.point_cloud_data(pcd_points.invert_point_cloud())
#o3d.visualization.draw_geometries([inverted_pcd.pcd])

In [ ]:
# Initilizing the CSF parameters
csf_params = csf.CSF_PARAMS(False, 0.1, 0.5, 0.5, 2.5, 1000, 0.5)

csf_filter = csf.CSF(csf_params)

cl = csf_filter.do_filtering(inverted_pcd.pcd, inverted_pcd.get_bounding_XY(), True)

print(">>>>>>>>> Filtering Completed.....")   

In [ ]:
ground_pcd = o3d.geometry.PointCloud()
ground_pcd = inverted_pcd.pcd.select_by_index(csf_filter.ground_points)
ground_pcd.paint_uniform_color([1, 0, 0])

In [ ]:
non_ground_pcd = o3d.geometry.PointCloud()
non_ground_pcd = inverted_pcd.pcd.select_by_index(csf_filter.non_ground_points)
non_ground_pcd.paint_uniform_color([0, 0, 0])

In [ ]:
o3d.visualization.draw_geometries([non_ground_pcd])

In [ ]:
cloth_pcd_list = []
num_particles = len(cl.cloth_parameters.particles)
for i in range(num_particles):
    cloth_pcd_list.append(cl.cloth_parameters.particles[i].get_position())

cloth_data = point_cloud.point_cloud_formatting(None)
cloth_data.downsample = False
cloth_pcd = cloth_data.o3d_format_from_array(np.asarray(cloth_pcd_list))
cloth_pcd.paint_uniform_color([0.6, 0.6, 0.6])

o3d.visualization.draw_geometries([cloth_pcd, inverted_pcd.pcd])

In [ ]:
o3d.visualization.draw_geometries([cloth_pcd, non_ground_pcd])

In [ ]:
# Heat Map
x = np.asarray(non_ground_pcd.points)[:1000, 0]
y = np.asarray(non_ground_pcd.points)[:1000, 1]
z = np.reshape(np.asarray(non_ground_pcd.points)[:1000, 2], (-1, 1))



plt.figure(figsize=(10, 10))
plt.contour(x, y, z, 20, cmap='RdGy')
plt.imshow(data, cmap='hot', interpolation='nearest')
plt.title( "Heatmap Sample (Syeilendra ITB)", fontsize=15 )
plt.xlabel("X Label", fontsize=10)
plt.ylabel("Y Label", fontsize=10)
plt.colorbar()
plt.show()

#pcd_2d = np.asarray(non_ground_pcd.points)[:, :2]
#plt.plot(pcd_2d[:, 0], pcd_2d[:, 1], '.', markersize = 0.1)

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.1, min_samples=10).fit(pcd_2d)
labels = db.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print("Estimated number of clusters: %d" % n_clusters)

unique_labels = set(labels)
  

In [ ]:
from scipy.spatial import ConvexHull
hull = ConvexHull(pcd_2d)
for simplex in hull.simplices:
    plt.plot(pcd_2d[simplex, 0], pcd_2d[simplex, 1], 'k-')
#plt.plot(pcd_2d[hull.simplices,0], pcd_2d[hull.simplices,1], 'r--', lw=2)
#plt.plot(1.05 * pcd_2d[hull.vertices,0], 1.05 * pcd_2d[hull.vertices,1], 'r--', lw=2)

In [ ]:
cloth_pcd.paint_uniform_color([0.6, 0.6, 0.6])
cloth_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(cloth_pcd, 0.1)

o3d.visualization.draw_geometries([cloth_mesh, inverted_pcd.pcd])

In [ ]:
np.asarray(cloth_pcd.points)

In [ ]:
# Comparing distance threshold
dist = pcd_inverted.compute_point_cloud_distance(cloth_pcd)
dist = np.asarray(dist)
ind  = np.where(dist > csf_params.height_threshold)[0]

non_ground = pcd_points.select_by_index(ind)
non_ground.paint_uniform_color([0.6, 0.6, 0.6])
ground = pcd_points.select_by_index(ind, invert=True)
ground.paint_uniform_color([0, 0, 0])
o3d.visualization.draw_geometries([non_ground])

In [ ]:

nearest_pc_points_id = []
for i in range(num_particles):
    nearest_pc_points_id.append(cl.cloth_parameters.particles[i].nearest_point_index)

ground_id = []
[ground_id.extend(x) for x in nearest_pc_points_id if (x != [] and x not in ground_id)]